In [52]:
!pip install langchain
!pip install -U langchain-community
pip install transformers
!pip install faiss-gpu
!pip install pypdf
!pip install boto3
!pip install streamlit
!pip install langchain_ollama
!pip install ngrok
!pip install pyngrok
!pip install -U langchain-huggingface
pip install absl-py

In [53]:
import json
import os
import sys
import boto3
import streamlit as st

import threading
import time
import os
import asyncio
import threading
import queue
import time
from threading import Thread

In [54]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from pyngrok import ngrok
from langchain_community.llms import Ollama
## LLm Models

from langchain.prompts import PromptTemplate



In [55]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
#loading the embedding model from huggingface
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

In [57]:
! curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 CLI
############################################################################################# 100.0%
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [58]:
# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
token="2elBx9gqMRNbecChxBY1LCFlimJ_3Gk8GJwiiRTynhLUUsF8a"
ngrok.set_auth_token(token)

# set up a stoppable thread (not mandatory, but cleaner if you want to stop this later
class StoppableThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

def start_ngrok(q, stop_event):
    try:
        # Start an HTTP tunnel on the specified port
        public_url = ngrok.connect(11434)
        # Put the public URL in the queue
        q.put(public_url)
        # Keep the thread alive until stop event is set
        while not stop_event.is_set():
            time.sleep(1)  # Adjust sleep time as needed
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

In [59]:
import os
import asyncio
# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it



In [60]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()


>>> starting ollama serve


In [50]:
!export OLLAMA_HOST=https://d12f-35-223-4-31.ngrok-free.app/

In [51]:
!ollama pull llama3

Error: could not connect to ollama app, is it running?


In [61]:
# Initialize an instance of the Ollama model
#llm = Ollama(model="llama3")

In [62]:
## Data ingestion
def data_ingestion():
    #Load document using PyPDFLoader document loader
    loader = PyPDFLoader("/content/drive/MyDrive/Invoice copy.pdf")
    documents = loader.load()
    #Splitting the data into chunk
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
    docs = text_splitter.split_documents(documents=documents)
    return docs

In [63]:
## Vector Embedding and vector store

def get_vector_store(docs):
    vectorstore_faiss=FAISS.from_documents(docs, embeddings)
    #retriever_faiss=vectorstore_faiss.as_retriever()

    #vectorstore = FAISS.from_documents(docs, embeddings)
    vectorstore_faiss.save_local("faiss_index")

In [64]:
    # Load from local storage
    #persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings,allow_dangerous_deserialization=True)

In [65]:
    #creating a retriever on top of database
    #retriever = persisted_vectorstore.as_retriever()


In [66]:
def get_llama3_llm():
    #Initialize an instance of the Ollama model
    llm = Ollama(model="llama3")
    return llm

In [67]:
prompt_template = """

Human: Use the following pieces of context to provide a
concise answer to the question at the end but usse atleast summarize with
250 words with detailed explaantions. If you don't know the answer,
just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [68]:
def get_response_llm(llm,vectorstore_faiss,query):
    qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
    answer=qa({"query":query})
    return answer['result']


In [69]:
def main():
    st.set_page_config("Chat PDF")

    st.header("Chat with PDF using Llama3💁")

    user_question = st.text_input("Ask a Question from the PDF Files")

    with st.sidebar:
        st.title("Update Or Create Vector Store:")

        if st.button("Vectors Update"):
            with st.spinner("Processing..."):
                docs = data_ingestion()
                get_vector_store(docs)
                st.success("Done")


    if st.button("Llama2 Output"):
        with st.spinner("Processing..."):
            faiss_index = FAISS.load_local("faiss_index", embeddings)
            llm=get_llama3_llm()

            #faiss_index = get_vector_store(docs)
            st.write(get_response_llm(llm,faiss_index,user_question))
            st.success("Done")

if __name__ == "__main__":
    main()




In [70]:
# Invoke the model to generate responses
#response = llm.invoke("Tell me a joke")
#print(response)

In [71]:
!ipython -m streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py RAG_With_Llama_pdf.ipynb

]0;IPython: drive/MyDrive


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.197.62:8501

  Stopping...
  Stopping...
Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <IPython.terminal.interactiveshell.TerminalInteractiveShell object at 0x780aa89467d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3924, in atexit_operations
    self.reset(new_session=False)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 1456, in reset
    self.displayhook.flush()
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/displayhook.py", line 311, in flush
    gc.collect()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/bootstrap.py", line 44, in signal_handler
    server.stop()
  File "/usr/local/lib/python3.10/dist-pa

In [76]:
!pip install ngrok
!ngrok http 8501 &
!ipython -m streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py RAG_With_Llama_pdf.ipynb




http - start an HTTP tunnel

USAGE:
  ngrok http [address:port | port] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080                        # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --domain mydomain.com                           # run ngrok with your own custom domain
  ngrok http 80 --allow-cidr 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions
  Upgrade your a